In [12]:
import json
import os
import re

# Function to extract mapping entries from a JSON file
def extract_mapping_entries(filename):
    # match = re.search(r'JSON_([a-zA-Z]+)_AssemblyElection_(\d{4})_A(\d{3}).json', filename)
    match = re.search(r"Form20-(\d+)", filename)

    if match:
        # state = match.group(1).lower()  # Convert state name to lowercase
        state = 'rajasthan'
        year = 2018
        AC = int(match.group(1))
    else:
        print(f"Filename {filename} does not match expected format.")
        return []

    with open(filename) as f:
        data = json.load(f)

    table = data['tables'][0]
    col_headers = []
    cells = table.get('cells')

    for i in range(table.get('column_count')):
        col_cells = [cell for cell in cells if cell.get('column_index') == i and cell.get('kind') == 'columnHeader' and cell.get('column_span') == 1 and cell.get('row_span') == 1]
                
        contents = [cell.get('content') for cell in col_cells]
                
        contents = [content for content in contents if re.search(r'[a-zA-Z]', content) and len(content) > 4]
        content = contents[0] if len(contents) else None
        if content: 
            col_headers.append({
                "name": content.replace('\n', ' ').strip(),
                "index" : i + 1 
            })

    mapping_entries = []
    for column in col_headers:
        mapping_entry = {
            "state": state,
            "year": year,
            "AC": AC,
            "name": column.get('name'),
            "column": column.get('index'),
        }
        mapping_entries.append(mapping_entry)

    return mapping_entries


In [13]:
folder_path = 'results/Parsed_Pdfs/RA_filtered/2023'

final_mapping_entries = []

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        mapping_entries = extract_mapping_entries(file_path)
        final_mapping_entries.extend(mapping_entries)

state = 'rajasthan'
year = 2023
# Write final mapping entries to a JSON file
final_mapping_filename = f'results/name_mappings/{state}_{year}.json'

with open(final_mapping_filename, 'w') as f:
    json.dump(final_mapping_entries, f, indent=4)

print(f"Final mapping file created: {final_mapping_filename}")

Final mapping file created: results/name_mappings/rajasthan_2023.json
